<a href="https://colab.research.google.com/github/leomaurodesenv/kaggle/blob/main/jupyter/company_challenge-desafio_de_inteligencia_artificial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Company Challenge
**Código**: leomaurodesenv <leo.mauro.desenv@gmail.com>

A \<empresa\> tem o objetivo de desburocratizar os serviços cartorários para nossos clientes B2B, realizando a busca, gestão e pré-análise dos documentos, assim reduzindo o tempo de entrega e acompanhando o andamento de seu pedido através do nosso produto.

## Desafio

Atualmente, nossos clientes solicitam ou nos enviam vários tipos de documentos. A partir desses documentos, extraímos e selecionamos diversas informações importantes para eles. Para que essas informações sejam corretamente extraídas, um processo automático de classificação do tipo de documento é necessário.

O desafio é a classificação de pelo menos os documentos RG e CPF, usando suas imagens, através de aprendizagem de máquina (machine learning).

# Coleta e Produção do Dataset

In [1]:
%%capture
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# full dataset
# !gdown 1Oi88TRcpdjZmJ79WDLb9qFlBNG8q2De6

In [4]:
# sample dataset
!gdown 144EqqmMtCziua9iYo-3afUEvZrJVxUXU

Downloading...
From: https://drive.google.com/uc?id=144EqqmMtCziua9iYo-3afUEvZrJVxUXU
To: /content/BID Sample Dataset.zip
100% 71.1M/71.1M [00:00<00:00, 166MB/s]
